In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

# Overview

(heavily derived from rdeits/DetroitTechWatch2020.jl)

* What is Julia?
* Julia is fast!
* Get more science done with Julia

# What is Julia

https://docs.julialang.org/en/v1/

(hmm what's the best)

# The Basics

Arithmetic

In [11]:
2 + 3

5

Strings:

In [13]:
# Strings
print("hello DESC")

hello DESC

Arrays:

In [12]:
A = [1, 2, 3, 4]

4-element Vector{Int64}:
 1
 2
 3
 4

# Functions

In [5]:
function square(x)
    return x * x 
end

square (generic function with 1 method)

In [6]:
square(2)

4

In [7]:
square("hello")

"hellohello"

# Why is Julia fast?

Julia will compile native-machine code into memory, right when it is needed, also known as Just-In-Time (JIT) compilation.

In [10]:
square(2)

4

In [16]:
@code_native square(2)

	.text
; ┌ @ In[5]:1 within `square'
	movq	%rdi, %rax
; │ @ In[5]:2 within `square'
; │┌ @ int.jl:88 within `*'
	imulq	%rdi, %rax
; │└
	retq
	nopl	(%rax,%rax)
; └


These are x86 *hardware instructions*. This function has been compiled into the same form that a C or Fortran function would be.

# 